# RLDT

## Step 1: Import the necessary libraries:

In [27]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [28]:
num_IOT_devices = 5

voltages_frequencies_IOT = [
    (1e6, 1.8),
    (2e6, 2.3),
    (4e6, 2.7),
    (8e6, 4.0),
    (16e6, 5.0),
    (32e6, 6.5),
]
num_MEC_devices = 10

voltages_frequencies_MEC = [
    (6 * 1e8, 0.8),
    (7.5 * 1e8, 0.825),
    (10 * 1e8, 1.0),
    (15 * 1e8, 1.2),
    (30 * 1e8, 2),
    (40 * 1e8, 3.1),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 100

#### *IOT*

In [29]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_IOT[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "ISL": np.random.randint(10, 21),
        "capacitance": [np.random.uniform(2, 3) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([700, 800, 900]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,,
0,4,"[1, 0, 0, 0]","[(32000000.0, 6.5), (1000000.0, 1.8), (4000000...",15,"[2.2187642195730702e-09, 2.5581020020173416e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.800000e+10,0.01,"[2, 3]",1
1,6,"[1, 1, 0, 0, 1, 0]","[(16000000.0, 5.0), (32000000.0, 6.5), (800000...",19,"[2.985650454110601e-09, 2.2420552715115008e-09...","[0.0007, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.01,"[4, 3, 1, 2]",1
2,8,"[1, 0, 1, 0, 0, 0, 1, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (4000000...",17,"[2.1375209441459933e-09, 2.341066351050259e-09...","[0.0007, 0.0007, 0.0007999999999999999, 0.0007...",3.600000e+10,0.01,"[1, 2, 4, 3]",1
3,4,"[1, 1, 0, 0]","[(1000000.0, 1.8), (16000000.0, 5.0), (4000000...",18,"[2.6635017691080557e-09, 2.005061583846219e-09...","[0.0007999999999999999, 0.0009, 0.0007, 0.0007]",3.900000e+10,0.01,"[3, 4]",1
4,8,"[0, 0, 1, 0, 1, 1, 0, 0]","[(1000000.0, 1.8), (8000000.0, 4.0), (2000000....",20,"[2.434394365510429e-09, 2.3500784076946757e-09...","[0.0007, 0.0007, 0.0007999999999999999, 0.0007...",3.900000e+10,0.01,"[2, 3]",1


#### *MEC*

In [30]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_MEC[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([9, 9, 10]) * 1e-5 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,capacitance,powerIdle,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,
0,64,"[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, ...","[(600000000.0, 0.8), (750000000.0, 0.825), (40...","[1.7894324477537793e-09, 1.7192370615090437e-0...","[9e-05, 0.0001, 9e-05, 9e-05, 0.0001, 9e-05, 9...",0.05,"[1, 2, 3, 4]",0
1,16,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]","[(750000000.0, 0.825), (4000000000.0, 3.1), (3...","[1.9235715720477948e-09, 1.6774525952313605e-0...","[9e-05, 0.0001, 9e-05, 9e-05, 0.0001, 0.0001, ...",0.10,"[3, 4, 1, 2]",0
2,32,"[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, ...","[(3000000000.0, 2), (750000000.0, 0.825), (150...","[1.7343303209970632e-09, 1.5281516378409186e-0...","[9e-05, 9e-05, 9e-05, 9e-05, 9e-05, 9e-05, 9e-...",0.09,"[4, 2]",1
3,64,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[(1000000000.0, 1.0), (4000000000.0, 3.1), (30...","[1.9383268013291726e-09, 1.7017414331061988e-0...","[9e-05, 9e-05, 0.0001, 9e-05, 0.0001, 0.0001, ...",0.06,"[2, 1, 3, 4]",0
4,16,"[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]","[(4000000000.0, 3.1), (750000000.0, 0.825), (1...","[1.8484807311832109e-09, 1.998627767384197e-09...","[0.0001, 9e-05, 9e-05, 0.0001, 9e-05, 9e-05, 9...",0.10,"[3, 1, 2]",0
5,32,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[(3000000000.0, 2), (4000000000.0, 3.1), (7500...","[1.920914388379136e-09, 1.5698861883131449e-09...","[9e-05, 0.0001, 0.0001, 9e-05, 9e-05, 9e-05, 9...",0.06,"[2, 1, 4, 3]",0
6,32,"[0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, ...","[(1000000000.0, 1.0), (750000000.0, 0.825), (6...","[1.825683605494123e-09, 1.885623334259869e-09,...","[9e-05, 9e-05, 0.0001, 9e-05, 9e-05, 9e-05, 9e...",0.10,"[2, 3, 4]",0
7,64,"[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, ...","[(3000000000.0, 2), (750000000.0, 0.825), (600...","[1.6024921477079106e-09, 1.6465738651305067e-0...","[0.0001, 9e-05, 0.0001, 9e-05, 0.0001, 9e-05, ...",0.05,"[1, 3, 2, 4]",1
8,64,"[0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, ...","[(600000000.0, 0.8), (4000000000.0, 3.1), (300...","[1.7697456461876688e-09, 1.8953615824194817e-0...","[9e-05, 9e-05, 9e-05, 0.0001, 9e-05, 0.0001, 9...",0.08,"[1, 2]",0


#### *CLOUD*

In [31]:
cloud = [(2.8e9, 1), (3.9e9, 2)]

### Step 2.2: Application

#### *helper function : generate_random_dag*

In [32]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [33]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)

    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
            "dataEntrySize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "returnDataSize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)

tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
t430,"[t428, t429]",4,3,0,822771,9000000,3000000,READY
t1134,[t1125],1,3,0,694945,2000000,8000,READY
t297,"[t291, t295, t296]",3,3,0,677279,2000000,6000,READY
t593,[t591],7,2,0,664190,1000,9000,READY
t1088,"[t1077, t1084]",8,4,0,822435,9000000,2000000,READY
...,...,...,...,...,...,...,...,...
t293,[t291],6,1,0,545327,7000,3000000,READY
t899,"[t895, t896, t897, t898]",3,3,0,626349,9000,1000000,READY
t399,"[t391, t393, t394]",5,2,0,583521,9000000,6000000,READY


## Step 3: Preprocessing

### Step 3.1: Clustering

#### *Tasks*

In [34]:
x = 0
for kind in task_kinds:
    for safe in (0, 1):
        selected_tasks = tasks.loc[(tasks["kind"] == kind) & (tasks["safe"] == safe), :]
        x += 1
        tasks.loc[selected_tasks.index, "cluster"] = x
        
tasks["cluster"] = tasks["cluster"].astype(int)
tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status,cluster
id,,,,,,,,,
t430,"[t428, t429]",4,3,0,822771,9000000,3000000,READY,5
t1134,[t1125],1,3,0,694945,2000000,8000,READY,5
t297,"[t291, t295, t296]",3,3,0,677279,2000000,6000,READY,5
t593,[t591],7,2,0,664190,1000,9000,READY,3
t1088,"[t1077, t1084]",8,4,0,822435,9000000,2000000,READY,7
...,...,...,...,...,...,...,...,...,...
t293,[t291],6,1,0,545327,7000,3000000,READY,1
t899,"[t895, t896, t897, t898]",3,3,0,626349,9000,1000000,READY,5
t399,"[t391, t393, t394]",5,2,0,583521,9000000,6000000,READY,3


#### *IOT & MEC Devices*

In [35]:
x = 1
IoTdevices["cluster"] = [[] for _ in range(len(IoTdevices))]

for kind in task_kinds:
    for safe in (0, 1):
        selected_IoTdevices = IoTdevices[
            IoTdevices["accetableTasks"].apply(lambda lst: kind in lst)
        ]

        condition = (IoTdevices.index.isin(selected_IoTdevices.index)) & (
            (IoTdevices["handleSafeTask"] == 0) & (safe == 0)
            | (IoTdevices["handleSafeTask"] == 1)
        )

        IoTdevices.loc[condition, "cluster"] = IoTdevices.loc[
            condition, "cluster"
        ].apply(lambda lst: lst + [x])

        x += 1
# ---------------------------------
x = 1
MECDevices["cluster"] = [[] for _ in range(len(MECDevices))]

for kind in task_kinds:
    for safe in (0, 1):
        selected_MECDevices = MECDevices[
            MECDevices["accetableTasks"].apply(lambda lst: kind in lst)
        ]

        condition = (MECDevices.index.isin(selected_MECDevices.index)) & (
            (MECDevices["handleSafeTask"] == 0) & (safe == 0)
            | (MECDevices["handleSafeTask"] == 1)
        )

        MECDevices.loc[condition, "cluster"] = MECDevices.loc[
            condition, "cluster"
        ].apply(lambda lst: lst + [x])

        x += 1

MECDevices
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask,cluster
id,,,,,,,,,,,
0,4,"[1, 0, 0, 0]","[(32000000.0, 6.5), (1000000.0, 1.8), (4000000...",15,"[2.2187642195730702e-09, 2.5581020020173416e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.800000e+10,0.01,"[2, 3]",1,"[3, 4, 5, 6]"
1,6,"[1, 1, 0, 0, 1, 0]","[(16000000.0, 5.0), (32000000.0, 6.5), (800000...",19,"[2.985650454110601e-09, 2.2420552715115008e-09...","[0.0007, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.01,"[4, 3, 1, 2]",1,"[1, 2, 3, 4, 5, 6, 7, 8]"
2,8,"[1, 0, 1, 0, 0, 0, 1, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (4000000...",17,"[2.1375209441459933e-09, 2.341066351050259e-09...","[0.0007, 0.0007, 0.0007999999999999999, 0.0007...",3.600000e+10,0.01,"[1, 2, 4, 3]",1,"[1, 2, 3, 4, 5, 6, 7, 8]"
3,4,"[1, 1, 0, 0]","[(1000000.0, 1.8), (16000000.0, 5.0), (4000000...",18,"[2.6635017691080557e-09, 2.005061583846219e-09...","[0.0007999999999999999, 0.0009, 0.0007, 0.0007]",3.900000e+10,0.01,"[3, 4]",1,"[5, 6, 7, 8]"
4,8,"[0, 0, 1, 0, 1, 1, 0, 0]","[(1000000.0, 1.8), (8000000.0, 4.0), (2000000....",20,"[2.434394365510429e-09, 2.3500784076946757e-09...","[0.0007, 0.0007, 0.0007999999999999999, 0.0007...",3.900000e+10,0.01,"[2, 3]",1,"[3, 4, 5, 6]"


### Step 3.2: Queueing

In [36]:
sorted_tasks = tasks.sort_values(by="mobility").copy()
proccessingQueue = set()
for index, row in sorted_tasks.iterrows():
    if row["status"] == "Queued":
        continue
    for dep in row["dependency"]:
        if sorted_tasks.loc[dep, "status"] != "Queued":
            proccessingQueue.add(dep)
            sorted_tasks.loc[dep, "status"] = "Queued"
            dependecny_meet = False

    proccessingQueue.add(index)
    sorted_tasks.loc[index, "status"] = "Queued"

proccessingQueue = list(proccessingQueue)
proccessingQueue

['t484',
 't908',
 't621',
 't1014',
 't939',
 't1030',
 't42',
 't1093',
 't1195',
 't980',
 't7',
 't1134',
 't395',
 't236',
 't703',
 't464',
 't1129',
 't543',
 't544',
 't97',
 't644',
 't472',
 't1077',
 't1119',
 't739',
 't438',
 't267',
 't761',
 't150',
 't239',
 't1173',
 't1025',
 't553',
 't1124',
 't193',
 't479',
 't853',
 't251',
 't44',
 't111',
 't669',
 't870',
 't28',
 't1043',
 't713',
 't1057',
 't23',
 't299',
 't1048',
 't448',
 't102',
 't608',
 't1095',
 't626',
 't164',
 't56',
 't357',
 't206',
 't894',
 't750',
 't82',
 't262',
 't47',
 't706',
 't828',
 't565',
 't353',
 't503',
 't281',
 't1087',
 't541',
 't100',
 't376',
 't577',
 't1136',
 't730',
 't1060',
 't445',
 't309',
 't65',
 't1008',
 't1038',
 't540',
 't407',
 't1176',
 't310',
 't555',
 't1183',
 't709',
 't179',
 't29',
 't1104',
 't860',
 't11',
 't776',
 't928',
 't297',
 't1149',
 't592',
 't536',
 't1107',
 't457',
 't1162',
 't406',
 't775',
 't441',
 't137',
 't209',
 't566',
 't801

## Step 3: Setting Up The tree

In [38]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def calculate_Pn_per_node(weights,bias,values):
    weights_transposed = weights.reshape(-1, 1)
    result = np.dot(weights_transposed, values)
    result -= bias
    return sigmoid(result)


class Node:
    def __init__(self, data,features):
        self.data = data
        self.left = None
        self.right = None
        self.wieghts = np.ones(len(data))
        self.features = features
        self.bias = 0
        mean = 0
        std_dev = 10
        num_categories = 8
        self.propapbilityDist = np.random.normal(mean, std_dev, num_categories)


def build_tree(node, max_depth):
    if max_depth == 0 or len(node.data) == 0:
        return node.propapbilityDist 

    left_data = node.data[calculate_Pn_per_node(node.weights,node.bias,node.data[features].values) <0.5]
    right_data = node.data[calculate_Pn_per_node(node.weights,node.bias,node.data[features].values) >=0.5]
    node.left = Node(left_data)
    node.right = Node(right_data)

    build_tree(node.left, max_depth - 1)
    build_tree(node.right, max_depth - 1)


def predict(node, record):
    if node.prediction is not None:
        return node.prediction

    if calculate_Pn_per_node(node.weights, node.bias, row) < 0.5:
        return predict(node.left, row)
    else:
        return predict(node.right, row)


# root = Node(train_data)
# build_tree(root, max_depth=3)
